<a href="https://colab.research.google.com/github/MaksimNikolin/Carservice/blob/main/object_detection_yolov7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## *Unpacking the archive with the dataset*

In [ ]:
from google.colab import drive
drive.mount('/content/drive') #connect to GoogleDrive

Mounted at /content/drive


In [ ]:
#unpacking training and validation datasets
%cd /content
!unzip -q /content/drive/MyDrive/Colab_Notebooks/object_detection_carservice/dataset.zip

/content


## *Cloning the repository*

In [ ]:
save_to_drive = False

if save_to_drive:
     %cd /content/drive/MyDrive
else:
     %cd /content

!git clone https://github.com/WongKinYiu/yolov7 #cloning the model from the site

/content
Cloning into 'yolov7'...
remote: Enumerating objects: 1191, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 1191 (delta 2), reused 6 (delta 2), pack-reused 1185
Receiving objects: 100% (1191/1191), 74.23 MiB | 33.60 MiB/s, done.
Resolving deltas: 100% (511/511), done.


## *Loading suggested weights or using our own weights*

In [ ]:
!wget -q https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7-tiny.pt
weight_path = 'yolov7-tiny.pt'
#weight_path = '/content/drive/MyDrive/yolov7-tiny.pt'

In [ ]:
#paths to folders with training and validation datasets, and class names
train_path = '/content/train'
val_path = '/content/valid'
classes = ['person','car','plate']

str_classes = ','.join(classes)
data_text = [f'train: {train_path}',
         f'val: {val_path}',
         f'nc: {len(classes)}',
         f'names: [{str_classes}]']
data_text = '\n'.join(data_text)

In [ ]:
if save_to_drive:
    path_to_yolo7 = '/content/drive/MyDrive/yolov7'
else:
    path_to_yolo7 = '/content/yolov7'

with open(f'{path_to_yolo7}/data/custom_data.yaml', 'w') as f:
    f.write(data_text)

with open(f'{path_to_yolo7}/cfg/training/yolov7-tiny.yaml') as f:
    cfg_text = f.readlines()

cfg_text[1] = f'nc: {len(classes)} # number of classes\n'
with open(f'{path_to_yolo7}/cfg/training/yolov7-tiny_custom.yaml', 'w') as f:
    cfg_text = f.writelines(cfg_text)

## *Launching training*

In [ ]:
model_name = 'test4' #if it gives errors that it cannot find the image along the path, then it is necessary to delete the cache files

if save_to_drive:
    %cd /content/drive/MyDrive/yolov7
else:
    %cd /content/yolov7/

!python train.py --workers 4 --device 0 --batch-size 16 --epochs 51 --data data/custom_data.yaml \
                 --img 640 640 --cfg cfg/training/yolov7-tiny_custom.yaml --weights $weight_path \
                 --name $model_name --hyp data/hyp.scratch.custom.yaml

/content/yolov7
2023-07-11 15:46:28.760181: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-11 15:46:29.669801: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
YOLOR 🚀 v0.1-126-g84932d7 torch 2.0.1+cu118 CUDA:0 (Tesla T4, 15101.8125MB)

Namespace(weights='yolov7-tiny.pt', cfg='cfg/training/yolov7-tiny_custom.yaml', data='data/custom_data.yaml', hyp='data/hyp.scratch.custom.yaml', epochs=51, batch_size=16, img_size=[640, 640], rect=False, resume=False, nosave=False, notest=False, noautoanchor=False, evolve=False, bucket='', cache_images=False, image_weights=False, device='0', multi_scale=False, single_cls=False, adam=False, sync_bn=False, local_rank=-1, workers=4, project='runs/train', entity=None, n

In [ ]:
#installing the libraries necessary to run the model
!pip install -q onnx
!pip install -q onnxruntime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 56.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 55.4 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.6
    Uninstalling protobuf-3.19.6:
      Successfully uninstalled protobuf-3.19.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.11.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 64.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 KB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
model_path = '/content/yolov7/runs/train/ppc1/weights/best.pt' #path to training model

In [ ]:
if save_to_drive:
    %cd /content/drive/MyDrive/yolov7
else:
    %cd /content/yolov7

#for onnxruntime, you need to specify this value as an integer, when it is 0 it means agnostic NMS, otherwise it is non-agnostic NMS
!python export.py --weights $model_path --grid --end2end --simplify \
                  --device 0 --topk-all 100 --iou-thres 0.3 \
                  --conf-thres 0.3 --img-size 224 416 --max-wh 0

/content/yolov7
Import onnx_graphsurgeon failure: No module named 'onnx_graphsurgeon'
Namespace(weights='/content/yolov7/runs/train/ppc1/weights/best.pt', img_size=[224, 416], batch_size=1, dynamic=False, dynamic_batch=False, grid=True, end2end=True, max_wh=0, topk_all=100, iou_thres=0.3, conf_thres=0.3, device='0', simplify=True, include_nms=False, fp16=False, int8=False)
YOLOR 🚀 v0.1-122-g3b41c2c torch 1.13.1+cu116 CUDA:0 (Tesla T4, 15101.8125MB)

Fusing layers... 
IDetect.fuse
Model Summary: 208 layers, 6013008 parameters, 0 gradients
/usr/local/lib/python3.9/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]

Starting TorchScript export with torch 1.13.1+cu116...
/content/yolov7/models/yolo.py:150: TracerWarning: Converting a tensor to a Python boolea

## *Model testing*

In [ ]:
#copy files from validation folder to testing folder
from posixpath import basename
import shutil
import numpy as np
import glob
import os

os.mkdir('/content/test_imgs')
for _ in range(10):
    img_path = np.random.choice(glob.glob(f'{val_path}/images/*'))
    filename = os.path.basename(img_path)
    dest_path = f'/content/test_imgs/{filename}'
    print(dest_path)
    shutil.copy(img_path, dest_path)
    #print(img_path)

In [ ]:
import cv2
import time
import requests
import random
import numpy as np
import onnxruntime as ort
from PIL import Image
from pathlib import Path
from collections import OrderedDict, namedtuple
import glob
import matplotlib.pyplot as plt
cuda = True

w = '/content/yolov7/runs/train/ppc1/weights/best.onnx'  #path to converted model from a cell with conversion
inp_w = 416 #permission to enter the converted model
inp_h = 224
names = ['car','person','plate'] #class names
SCORE_THRESH = 0.5

test_folder = '/content/test_imgs' #folder with testing pics

providers = ['CUDAExecutionProvider', 'CPUExecutionProvider'] if cuda else ['CPUExecutionProvider']
session = ort.InferenceSession(w, providers=providers)

def letterbox(im, new_shape=(640, 640), color=(114, 114, 114), auto=True, scaleup=True, stride=32):
    #resize and pad image while meeting stride-multiple constraints
    shape = im.shape[:2]  #current shape [height, width]
    if isinstance(new_shape, int):
        new_shape = (new_shape, new_shape)

    #scale ratio (new/old)
    r = min(new_shape[0] / shape[0], new_shape[1] / shape[1])
    if not scaleup:  #only scale down, do not scale up (for better val mAP)
        r = min(r, 1.0)

    #compute padding
    new_unpad = int(round(shape[1] * r)), int(round(shape[0] * r))
    dw, dh = new_shape[1] - new_unpad[0], new_shape[0] - new_unpad[1]  #wh padding

    if auto:  #minimum rectangle
        dw, dh = np.mod(dw, stride), np.mod(dh, stride)  #wh padding

    dw /= 2  #divide padding into 2 sides
    dh /= 2

    if shape[::-1] != new_unpad:  #resize
        im = cv2.resize(im, new_unpad, interpolation=cv2.INTER_LINEAR)
    top, bottom = int(round(dh - 0.1)), int(round(dh + 0.1))
    left, right = int(round(dw - 0.1)), int(round(dw + 0.1))
    im = cv2.copyMakeBorder(im, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)  #add border
    return im, r, (dw, dh)

colors = {name:[random.randint(0, 255) for _ in range(3)] for i,name in enumerate(names)}

In [ ]:
for img_path in glob.glob(f'{test_folder}/*'):
    img = cv2.imread(img_path)[246:960, 260:1590]
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    image = img.copy()
    image = cv2.resize(image, (inp_w, inp_h))
    image = image.transpose((2, 0, 1))
    image = np.expand_dims(image, 0)
    image = np.ascontiguousarray(image)

    im = image.astype(np.float32)
    im /= 255
    im.shape

    outname = [i.name for i in session.get_outputs()]
    inname = [i.name for i in session.get_inputs()]

    inp = {inname[0]:im}

    #ONNX inference
    outputs = session.run(outname, inp)[0]
    outputs

    ori_images = [img.copy()]

    for i,(batch_id,x0,y0,x1,y1,cls_id,score) in enumerate(outputs):
        if score < SCORE_THRESH:
            break
        image = ori_images[int(batch_id)]
        x0 = int(x0 / inp_w * img.shape[1])
        y0 = int(y0 / inp_h * img.shape[0])
        x1 = int(x1 / inp_w * img.shape[1])
        y1 = int(y1 / inp_h * img.shape[0])
        cls_id = int(cls_id)
        score = round(float(score),3)
        name = names[cls_id]
        color = colors[name]
        name += ' ' + str(score)
        cv2.rectangle(img,(x0,y0),(x1,y1),color,2)
        cv2.putText(img,name,(x0, y0 - 2),cv2.FONT_HERSHEY_SIMPLEX,0.75,[225, 255, 255],thickness=2)
    plt.figure(figsize=(15,10))
    plt.imshow(img)
    #Image.fromarray(ori_images[0])

In [ ]:
#loading the weights of the model and the converted model file
from google.colab import files
files.download(model_path)
files.download(w)